<a href="https://colab.research.google.com/github/simonyelisey/Sibur_Challenge_2021/blob/main/ods_pd.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

import os,gc
import tqdm
from typing import List

In [2]:
def read_parquet_dataset_from_local(path_to_dataset: str, start_from: int = 0, num_parts_to_read: int = 2, 
                                    columns: List[str] = None, verbose: bool = False) -> pd.DataFrame:
    """
    Читает ``num_parts_to_read партиций`` и преобразует их к pandas.DataFrame.
    Параметры:
    -----------
    path_to_dataset: str
        Путь до директории с партициями.
    start_from: int, default=0
        Номер партиции, с которой начать чтение.
    num_parts_to_read: int, default=2
        Число партиций, которые требуется прочитать.
    columns: List[str], default=None
        Список колонок, которые нужно прочитать из каждой партиции. Если None, то считываются все колонки.
    Возвращаемое значение:
    ----------------------
    frame: pandas.DataFrame
        Прочитанные партиции, преобразованные к pandas.DataFrame.
    """

    res = []
    start_from = max(0, start_from)
    # dictionory of format {partition number: partition filename}
    dataset_paths = {int(os.path.splitext(filename)[0].split("_")[-1]): os.path.join(path_to_dataset, filename)
                     for filename in os.listdir(path_to_dataset)}
    chunks = [dataset_paths[num] for num in sorted(dataset_paths.keys()) if num>=start_from][:num_parts_to_read]
    
    if verbose:
        print("Reading chunks:", *chunks, sep="\n")
    for chunk_path in tqdm.tqdm_notebook(chunks, desc="Reading dataset with pandas"):
        chunk = pd.read_parquet(chunk_path, columns=columns)
        res.append(chunk)
    return pd.concat(res).reset_index(drop=True)

In [3]:
%load_ext autoreload
%autoreload 2

import gc
import os
import sys
import pandas as pd
import numpy as np
import tqdm
import seaborn as sns


import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline  

from sklearn.model_selection import train_test_split, StratifiedKFold, KFold
from sklearn.metrics import roc_auc_score

pd.set_option('display.max_columns', None)

# если у вас есть CUDA, то она понадобится там для экспериментов в catboost
os.environ["CUDA_VISIBLE_DEVICES"] = '0'

sys.path.append('../')

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
os.chdir('/content/drive/MyDrive')

In [6]:
os.listdir('kaggle/datafest/data_for_competition')

['test_target.csv',
 'sample_submission.csv',
 'description.xlsx',
 'train_target.csv',
 'test_data',
 'train_data',
 'train_features_gb',
 'test_features_gb']

In [7]:
train = pd.read_parquet('kaggle/datafest/data_for_competition/train_data/train_data_0.pq')
train

,id,rn,pre_since_opened,pre_since_confirmed,pre_pterm,pre_fterm,pre_till_pclose,pre_till_fclose,pre_loans_credit_limit,pre_loans_next_pay_summ,pre_loans_outstanding,pre_loans_total_overdue,pre_loans_max_overdue_sum,pre_loans_credit_cost_rate,pre_loans5,pre_loans530,pre_loans3060,pre_loans6090,pre_loans90,is_zero_loans5,is_zero_loans530,is_zero_loans3060,is_zero_loans6090,is_zero_loans90,pre_util,pre_over2limit,pre_maxover2limit,is_zero_util,is_zero_over2limit,is_zero_maxover2limit,enc_paym_0,enc_paym_1,enc_paym_2,enc_paym_3,enc_paym_4,enc_paym_5,enc_paym_6,enc_paym_7,enc_paym_8,enc_paym_9,enc_paym_10,enc_paym_11,enc_paym_12,enc_paym_13,enc_paym_14,enc_paym_15,enc_paym_16,enc_paym_17,enc_paym_18,enc_paym_19,enc_paym_20,enc_paym_21,enc_paym_22,enc_paym_23,enc_paym_24,enc_loans_account_holder_type,enc_loans_credit_status,enc_loans_credit_type,enc_loans_account_cur,pclose_flag,fclose_flag
0,0,1,18,9,2,3,16,10,11,3,3,0,2,11,6,16,5,4,8,1,1,1,1,1,16,2,17,1,1,1,0,0,3,3,3,3,3,3,3,3,3,4,3,3,3,3,3,3,3,3,4,3,3,3,4,1,3,4,1,0,0
1,0,2,18,9,14,14,12,12,0,3,3,0,2,11,6,16,5,4,8,1,1,1,1,1,16,2,17,1,1,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,4,1,3,4,1,0,0
2,0,3,18,9,4,8,1,11,11,0,5,0,2,8,6,16,5,4,8,1,1,1,1,1,15,2,17,0,1,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,4,1,2,3,1,1,1
3,0,4,4,1,9,12,16,7,12,2,3,0,2,4,6,16,5,4,8,0,1,1,1,1,16,2,17,1,1,1,1,0,0,0,0,0,0,0,0,0,0,1,3,3,3,3,3,3,3,3,4,3,3,3,4,1,3,1,1,0,0
4,0,5,5,12,15,2,11,12,10,2,3,0,2,4,6,16,5,4,8,1,1,1,1,1,16,2,17,1,1,1,0,0,0,0,0,0,0,3,3,3,3,4,3,3,3,3,3,3,3,3,4,3,3,3,4,1,3,4,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1974719,249998,4,1,9,14,8,10,11,8,4,2,0,2,4,6,16,5,4,8,1,1,1,1,1,11,2,17,0,1,1,0,0,0,0,0,0,0,0,0,0,0,1,0,3,3,3,3,3,3,3,4,3,3,3,4,1,2,3,1,0,1
1974720,249998,5,7,9,4,8,1,11,19,1,4,0,2,7,6,16,5,4,8,1,1,1,1,1,7,2,17,0,1,1,0,0,0,0,0,3,3,3,3,3,3,4,3,3,3,3,3,3,3,3,4,3,3,3,4,1,2,4,1,1,1
1974721,249999,1,9,0,10,8,10,11,16,2,3,0,2,7,6,16,5,4,8,1,1,1,1,1,16,2,17,1,1,1,0,0,0,0,0,0,0,0,3,3,3,4,3,3,3,3,3,3,3,3,4,3,3,3,4,1,2,3,1,0,1
1974722,249999,2,9,16,10,13,10,4,12,2,3,0,2,7,6,16,5,4,8,1,1,1,1,1,16,2,17,1,1,1,0,0,0,0,0,0,0,0,0,3,3,4,3,3,3,3,3,3,3,3,4,3,3,3,4,1,2,3,1,0,0


In [13]:
test = pd.read_parquet('kaggle/datafest/data_for_competition/test_data/test_data_0.pq')
test

,id,rn,pre_since_opened,pre_since_confirmed,pre_pterm,pre_fterm,pre_till_pclose,pre_till_fclose,pre_loans_credit_limit,pre_loans_next_pay_summ,pre_loans_outstanding,pre_loans_total_overdue,pre_loans_max_overdue_sum,pre_loans_credit_cost_rate,pre_loans5,pre_loans530,pre_loans3060,pre_loans6090,pre_loans90,is_zero_loans5,is_zero_loans530,is_zero_loans3060,is_zero_loans6090,is_zero_loans90,pre_util,pre_over2limit,pre_maxover2limit,is_zero_util,is_zero_over2limit,is_zero_maxover2limit,enc_paym_0,enc_paym_1,enc_paym_2,enc_paym_3,enc_paym_4,enc_paym_5,enc_paym_6,enc_paym_7,enc_paym_8,enc_paym_9,enc_paym_10,enc_paym_11,enc_paym_12,enc_paym_13,enc_paym_14,enc_paym_15,enc_paym_16,enc_paym_17,enc_paym_18,enc_paym_19,enc_paym_20,enc_paym_21,enc_paym_22,enc_paym_23,enc_paym_24,enc_loans_account_holder_type,enc_loans_credit_status,enc_loans_credit_type,enc_loans_account_cur,pclose_flag,fclose_flag
0,3000000,1,11,5,17,14,12,11,3,2,3,0,2,2,6,16,5,4,8,1,0,1,1,1,16,2,17,1,1,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,3,3,3,4,3,3,3,4,1,3,4,1,0,0
1,3000000,2,19,16,15,9,12,11,16,3,2,0,2,8,6,16,5,4,8,1,1,1,1,1,15,2,17,0,1,1,0,0,0,0,3,3,3,3,3,3,3,4,3,3,3,3,3,3,3,3,4,3,3,3,4,1,2,4,1,0,0
2,3000001,1,16,17,8,5,4,9,5,2,3,0,2,4,6,16,5,4,8,1,0,0,1,1,16,2,17,1,1,1,0,2,1,0,1,0,0,3,3,3,3,4,3,3,3,3,3,3,3,3,4,3,3,3,4,1,3,4,1,0,0
3,3000001,2,16,7,9,0,4,9,1,2,3,0,2,4,6,16,5,4,8,1,0,0,1,1,16,2,17,1,1,1,0,2,1,2,2,2,2,1,0,0,0,4,3,3,3,3,3,3,3,3,4,3,3,3,4,1,3,4,1,0,0
4,3000001,3,10,0,14,7,11,12,2,2,3,0,2,4,3,16,5,4,8,0,1,0,0,1,16,2,17,1,1,1,0,0,1,1,1,1,1,1,3,2,1,2,1,1,1,1,1,1,0,1,2,1,1,0,2,1,3,4,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2389768,3249999,2,5,12,16,9,11,12,10,2,3,0,2,1,6,16,5,4,8,1,1,1,1,1,16,2,17,1,1,1,0,0,0,0,3,3,3,3,3,3,3,4,3,3,3,3,3,3,3,3,4,3,3,3,4,1,3,4,1,0,0
2389769,3249999,3,3,17,17,6,12,0,4,2,3,0,2,4,6,16,5,4,8,1,1,1,1,1,9,5,4,0,0,0,0,0,0,0,0,0,0,3,3,3,3,4,3,3,3,3,3,3,3,3,4,3,3,3,4,1,3,3,1,0,0
2389770,3249999,4,11,9,4,8,1,11,12,2,3,0,2,2,6,16,5,4,8,1,1,1,1,1,16,2,17,1,1,1,0,0,0,0,0,0,0,0,0,0,0,1,0,3,3,3,3,3,3,3,4,3,3,3,4,1,2,4,1,1,1
2389771,3249999,5,11,6,12,10,1,11,8,4,4,0,2,9,6,16,5,4,8,1,1,1,1,1,8,2,17,0,1,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,4,3,3,3,4,1,2,4,1,0,0


In [ ]:
train.memory_usage(index=True).sum() / 10**9

0.126382464

In [14]:
test.memory_usage(index=True).sum() / 10**9

0.1529456

Изменим формат колонок до int8

In [ ]:
# train
for num in range(12):  
  data = pd.read_parquet(f'kaggle/datafest/data_for_competition/train_data/train_data_{num}.pq')
  for col in data.columns[1:]:
    data[col] = data[col].astype('int8')
  data['id'] = data['id'].astype('int32')
  data.to_parquet(f'kaggle/datafest/data_for_competition/train_data/train_data_{num}.pq')

del data
gc.collect()

0

In [11]:
# test
for num in range(2):  
  data = pd.read_parquet(f'kaggle/datafest/data_for_competition/test_data/test_data_{num}.pq')
  for col in data.columns[1:]:
    data[col] = data[col].astype('int8')
  data['id'] = data['id'].astype('int32')
  data.to_parquet(f'kaggle/datafest/data_for_competition/test_data/test_data_{num}.pq')

del data
gc.collect()

0

In [ ]:
nunique_ids = 0
for num in range(12):  
  data = pd.read_parquet(f'kaggle/datafest/data_for_competition/train_data/train_data_{num}.pq')
  nunique_ids += data['id'].nunique()

print(f'There are {nunique_ids} unique ids in dataset.')

del data, nunique_ids
gc.collect()

There are 3000000 unique ids in dataset.


353

In [ ]:
train_unique_ids = []
for num in range(12):  
  data = pd.read_parquet(f'kaggle/datafest/data_for_competition/train_data/train_data_{num}.pq')
  train_unique_ids.append(data['id'].unique())

print(len(train_unique_ids))

test_unique_ids = []
for num in range(2):
  data = pd.read_parquet(f'kaggle/datafest/data_for_competition/train_data/train_data_{num}.pq')
  test_unique_ids.append(data['id'].unique())

print(len(test_unique_ids))

del data
gc.collect()

12
2


777

In [ ]:
for i in train_unique_ids:
  print(i.min(), i.max())

0 249999
250000 499999
500000 749999
750000 999999
1000000 1249999
1250000 1499999
1500000 1749999
1750000 1999999
2000000 2249999
2250000 2499999
2500000 2749999
2750000 2999999


In [ ]:
for i in test_unique_ids:
  print(i.min(), i.max())

0 249999
250000 499999


In [15]:
train_target = pd.read_csv('kaggle/datafest/data_for_competition/train_target.csv')
train_target.shape

(3000000, 2)

In [16]:
train_target.memory_usage(index=True).sum() / 10**9

0.048000128

In [ ]:
# id == 250000

,id,rn,pre_since_opened,pre_since_confirmed,pre_pterm,pre_fterm,pre_till_pclose,pre_till_fclose,pre_loans_credit_limit,pre_loans_next_pay_summ,...,enc_paym_21,enc_paym_22,enc_paym_23,enc_paym_24,enc_loans_account_holder_type,enc_loans_credit_status,enc_loans_credit_type,enc_loans_account_cur,pclose_flag,fclose_flag
0,250000,1,8,7,7,2,4,9,5,2,...,3,3,3,4,1,3,4,1,0,0
1,250000,2,14,8,12,10,6,3,12,6,...,3,3,3,4,1,3,4,1,0,0
2,250000,3,10,4,8,11,6,13,5,2,...,3,3,3,4,1,3,4,1,0,0
3,250000,4,13,12,8,6,13,5,12,2,...,3,3,3,4,1,3,4,1,0,0
4,250000,5,6,12,4,14,1,7,4,2,...,3,3,3,4,1,3,1,1,1,0
5,250000,6,18,10,0,8,8,11,7,2,...,0,0,0,1,1,2,3,1,0,1
6,250000,7,11,9,4,8,1,11,0,5,...,3,3,3,4,1,2,3,1,1,1


In [ ]:
TRAIN_DATA_PATH = "kaggle/datafest/data_for_competition/train_data/"
TEST_DATA_PATH = "kaggle/datafest/data_for_competition/test_data/"

TRAIN_TARGET_PATH = "kaggle/datafest/data_for_competition/train_target.csv"

In [ ]:
TRAIN_FEATURES_PATH = "kaggle/datafest/data_for_competition/train_features_gb/"
TEST_FEATURES_PATH = "kaggle/datafest/data_for_competition/test_features_gb/"

In [ ]:
def process_and_feature_engineer(df):
    # FEATURE ENGINEERING FROM 
    # https://www.kaggle.com/code/huseyincot/amex-agg-data-how-it-created
    all_cols = [c for c in list(df.columns) if c not in ['customer_ID','S_2']]
    cat_features = ["B_30","B_38","D_114","D_116","D_117","D_120","D_126","D_63","D_64","D_66","D_68"]
    num_features = [col for col in all_cols if col not in cat_features]

    test_num_agg = df.groupby("customer_ID")[num_features].agg(['mean', 'std', 'min', 'max', 'last'])
    test_num_agg.columns = ['_'.join(x) for x in test_num_agg.columns]

    test_cat_agg = df.groupby("customer_ID")[cat_features].agg(['count', 'last', 'nunique'])
    test_cat_agg.columns = ['_'.join(x) for x in test_cat_agg.columns]

    df = cudf.concat([test_num_agg, test_cat_agg], axis=1)
    del test_num_agg, test_cat_agg
    print('shape after engineering', df.shape )
    
    return df
  

In [ ]:
%%time


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:28: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


Transforming sequential credit data:   0%|          | 0/3 [00:00<?, ?it/s]

Reading chunks:
kaggle/datafest/data_for_competition/train_data/train_data_0.pq
kaggle/datafest/data_for_competition/train_data/train_data_1.pq
kaggle/datafest/data_for_competition/train_data/train_data_2.pq
kaggle/datafest/data_for_competition/train_data/train_data_3.pq


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:30: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


Reading dataset with pandas:   0%|          | 0/4 [00:00<?, ?it/s]